In [132]:
import pandas as pd
import requests
import json
import re
from unidecode import unidecode
import unicodedata
import pyarrow
import pyarrow.parquet as pq
from datetime import datetime, timedelta

In [206]:
url = 'https://api.openbrewerydb.org/breweries'

pg = 1

breweriesDF = pd.DataFrame()

while True:
    #print(f'Page: {pg}')
    params = {
        'page': pg,
        'per_page': 200
    }

    resp = requests.request('GET', url = url, params = params)
    resp.encoding = 'utf-8'
    beweries = json.loads(resp.content.decode('utf-8', errors='ignore'))

    if len(beweries) > 0:
         
        breweriesDF = pd.concat([breweriesDF, pd.DataFrame(beweries)])

        pg += 1

    else:
        break

In [208]:
breweriesDF.shape

breweriesDF

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,micro,1716 Topeka St,None,None,Norman,Oklahoma,73069-8224,United States,-97.46818222,35.25738891,4058160490,http://www.405brewing.com,Oklahoma,1716 Topeka St
1,9c5a66c8-cc13-416f-a5d9-0a769c87d318,(512) Brewing Co,micro,407 Radam Ln Ste F200,None,None,Austin,Texas,78745-1197,United States,None,None,5129211545,http://www.512brewing.com,Texas,407 Radam Ln Ste F200
2,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,1 of Us Brewing Company,micro,8100 Washington Ave,None,None,Mount Pleasant,Wisconsin,53406-3920,United States,-87.88336350209435,42.72010826899558,2624847553,https://www.1ofusbrewing.com,Wisconsin,8100 Washington Ave
3,ef970757-fe42-416f-931d-722451f1f59c,10 Barrel Brewing Co,large,1501 E St,None,None,San Diego,California,92101-6618,United States,-117.129593,32.714813,6195782311,http://10barrel.com,California,1501 E St
4,6d14b220-8926-4521-8d19-b98a2d6ec3db,10 Barrel Brewing Co,large,62970 18th St,None,None,Bend,Oregon,97701-9847,United States,-121.281706,44.08683531,5415851007,http://www.10barrel.com,Oregon,62970 18th St
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,57a70f5c-a995-4be1-949c-99244450940f,플레이그라운드 브루어리(playground Brewery),brewpub,"246-13, Isanpo-gil",Ilsanseo-gu,None,Goyang-si,Gyeonggido,10203,South Korea,126.7020113,37.66779467,031-912-2463,https://www.playgroundbrewery.com/,Gyeonggido,"246-13, Isanpo-gil"
151,64a4f65c-47a9-4d8e-ad2e-3cc898b9689c,핸드앤몰트 (Hand and Malt),brewpub,"19, Hangang-daero 38ga-gil",None,None,Yongsan-gu,Seoul,04386,South Korea,126.9705831,37.52985655,070-7178-4011,http://handandmalt.com/,Seoul,"19, Hangang-daero 38ga-gil"
152,a9541aa9-5c12-469b-8cbb-01fb5619cefc,허심청브루잉(Hursimchung Bru),brewpub,"23, Geumganggongwon-ro 20beon-gil",None,None,Dongnae-gu,Busan,47709,South Korea,129.0826027,35.21990206,051-550-2345,https://www.hotelnongshim.com/kr/index.php?pCo...,Busan,"23, Geumganggongwon-ro 20beon-gil"
153,6f317bdc-458e-466f-bd2a-9ab398d46631,헤이스탁 브루어리(Haystack Brewery),brewpub,"25, Pangyogongwon-ro 3-gil",Bundang-gu,None,Seongnam-si,Gyeonggido,13477,South Korea,127.0890607,37.39125615,070-7631-1110,None,Gyeonggido,"25, Pangyogongwon-ro 3-gil"


In [129]:
country = breweriesDF[breweriesDF['country'] == 'Austria' ]

country

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
10,b04c9621-bb87-439d-a499-94cb432bde90,Cafe Luis,bar,Weizberg 7,None,None,Weiz,Steiermark,8160,Austria,15.6358280445545,47.22378922896473,+4331723924,None,Steiermark,Weizberg 7
12,9355bc7c-bdc7-40c3-8932-e840c996ed86,Caf� Okei,bar,Feldkirchenstra�e 40,None,None,Kalsdorf bei Graz,Steiermark,8401,Austria,15.4604202756149,46.97131912786789,+43313557156,https://okei.at,Steiermark,Feldkirchenstra�e 40
124,16fb2c6e-c7e1-4d46-b9d0-1c86816eb129,Die Bierbotschaft,brewpub,Ponigler Str. 52,None,None,Wundschuh,Steiermark,8142,Austria,15.455186813460086,46.91291590525785,+436763530560,https://www.bierbotschaft.at,Steiermark,Ponigler Str. 52
50,fcd9312c-7531-4801-b481-52f4267cf628,Gösser Brauerei (Heineken),large,Brauhausgasse 1,None,None,Leoben,Steiermark,8700,Austria,15.09334420582467,47.362557227478526,+438102069791,https://www.goesser.at,Steiermark,Brauhausgasse 1
4,867eee9e-2138-4e62-97a9-b25e7fd1a1ff,Klosterbrauerei Stift Engelszell,nano,Stiftstra�e 6,None,None,Engelhartszell,Oberösterreich,4090,Austria,13.73460683956955,48.498616150950205,+4377178010,https://www.stift-engelszell.at/trappistenbrau...,Oberösterreich,Stiftstra�e 6
98,524ab865-1df1-4655-b19f-0eb427cfa0d6,Murauer Brauerei,large,Raffaltpl. 19,None,None,Murau,Steiermark,8850,Austria,14.165060917014614,47.11101605760986,+43353232660,https://www.murauerbier.at,Steiermark,Raffaltpl. 19
113,56758247-521c-4219-afa2-0665c48c50ef,Puntigamer Brauerei (Heineken),large,Triester Str. 361,None,None,Graz,Steiermark,8055,Austria,15.432570096538111,47.029627917574615,+4331629710013,https://www.puntigamer.at,Steiermark,Triester Str. 361
88,eb82907e-7f0d-4453-b07e-4e00763e1cf6,Schladminger Brauerei (Heineken),large,Hammerfeldweg 163,None,None,Schladming,Steiermark,8970,Austria,13.685049237310041,47.390297330890185,+433687225910,https://www.schladmingerbier.at,Steiermark,Hammerfeldweg 163
92,c88759b6-88d6-4172-bed7-61dec61e50aa,Schleppe Brauerei (Heineken),large,Schleppe-Platz 1,None,None,Klagenfurt am W�rthersee,K�rnten,9020,Austria,14.286632028634074,46.6416966591789,+4346342700,http://www.schleppe.at,K�rnten,Schleppe-Platz 1
100,4c8dff75-fa8c-41ce-9999-1f063b59addb,Schwechater Brauerei (Heineken),large,Mautner-Markhof-Stra�e 11,None,None,Schwechat,Nieder�sterreich,2320,Austria,16.467172715971294,48.14492358439182,+431701400,https://www.schwechater.at,Nieder�sterreich,Mautner-Markhof-Stra�e 11


In [62]:
#breweriesDF.dtypes
print(breweriesDF.isna().sum())

id                   0
name                 0
brewery_type         0
address_1          764
address_2         8247
address_3         8320
city                 0
state_province       0
postal_code          0
country              0
longitude         2328
latitude          2328
phone              851
website_url       1189
state                0
street             764
dtype: int64


In [8]:
breweriesDF.reset_index(drop=True, inplace=True)

O Parâmetro lines=True em df.to_json() e pd.read_json()
Quando você usa lines=True com df.to_json():
Se você usar lines=True ao escrever o JSON com o Pandas (df.to_json()), ele cria um arquivo no formato JSON Lines. O formato JSON Lines é diferente do JSON tradicional, porque ele armazena um objeto JSON por linha no arquivo. Em vez de um único array de objetos JSON, cada linha é um objeto separado, tornando o arquivo muito mais eficiente em termos de leitura e escrita, especialmente com grandes volumes de dados.

In [ ]:
breweriesDF.to_json("api_json.json", orient='records', lines=True)


In [115]:
df = pd.read_json("api_json.json", lines=True)

In [41]:
dffiltered = df[df['name'].str.contains(r'\)', case=False)]

dffiltered

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,micro,1716 Topeka St,None,None,Norman,Oklahoma,73069-8224,United States,-97.468182,35.257389,4058160490,http://www.405brewing.com,Oklahoma,1716 Topeka St
1,9c5a66c8-cc13-416f-a5d9-0a769c87d318,(512) Brewing Co,micro,407 Radam Ln Ste F200,None,None,Austin,Texas,78745-1197,United States,NaN,NaN,5129211545,http://www.512brewing.com,Texas,407 Radam Ln Ste F200
146,90d1fa86-77d7-4a21-9818-e338fa6fdb96,583 양조장(583 Brewery),brewpub,"583-8, Uiam-ro",Janggye-myeon,None,Jangsu-gun,Jeollabukdo,55621,South Korea,127.620512,35.682717,063-353-5515,http://www.583beer.modoo.at,Jeollabukdo,"583-8, Uiam-ro"
653,80683fda-c79f-4dc1-941a-a2b4165487fe,Barley's Brewing Company (Ale House No. 1),brewpub,467 N High St,None,None,Columbus,Ohio,43215-2007,United States,-83.002767,39.971941,6142282537,http://www.barleysbrewing.com,Ohio,467 N High St
698,1e03e945-7418-4d87-9e61-ca95542a4580,Basic City Beer Co. (Southside RVA),micro,212 W 6th St,None,None,Richmond,Virginia,23224,United States,NaN,NaN,8044474735,https://www.basiccitybeer.com/rva-map,Virginia,212 W 6th St
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8350,57a70f5c-a995-4be1-949c-99244450940f,플레이그라운드 브루어리(playground Brewery),brewpub,"246-13, Isanpo-gil",Ilsanseo-gu,None,Goyang-si,Gyeonggido,10203,South Korea,126.702011,37.667795,031-912-2463,https://www.playgroundbrewery.com/,Gyeonggido,"246-13, Isanpo-gil"
8351,64a4f65c-47a9-4d8e-ad2e-3cc898b9689c,핸드앤몰트 (Hand and Malt),brewpub,"19, Hangang-daero 38ga-gil",None,None,Yongsan-gu,Seoul,04386,South Korea,126.970583,37.529857,070-7178-4011,http://handandmalt.com/,Seoul,"19, Hangang-daero 38ga-gil"
8352,a9541aa9-5c12-469b-8cbb-01fb5619cefc,허심청브루잉(Hursimchung Bru),brewpub,"23, Geumganggongwon-ro 20beon-gil",None,None,Dongnae-gu,Busan,47709,South Korea,129.082603,35.219902,051-550-2345,https://www.hotelnongshim.com/kr/index.php?pCo...,Busan,"23, Geumganggongwon-ro 20beon-gil"
8353,6f317bdc-458e-466f-bd2a-9ab398d46631,헤이스탁 브루어리(Haystack Brewery),brewpub,"25, Pangyogongwon-ro 3-gil",Bundang-gu,None,Seongnam-si,Gyeonggido,13477,South Korea,127.089061,37.391256,070-7631-1110,None,Gyeonggido,"25, Pangyogongwon-ro 3-gil"


In [46]:
with open("api_json.json", "r") as f:
    data = json.load(f)

    lista_dicionarios = [v['0'] for k, v in data.items()]
    print(f"Dictionary List: {lista_dicionarios}")

df = pd.DataFrame(data)


AttributeError: 'list' object has no attribute 'items'

In [116]:
df_read = pd.read_json("api_json.json")  # Não usa lines=True

df_read = df_read.astype({
            'id': 'string',
            'name': 'string',
            'brewery_type': 'string',
            'address_1': 'string',
            'address_2': 'string',
            'address_3': 'string',
            'city': 'string',
            'state_province': 'string',
            'postal_code': 'string',
            'country': 'string',
            'longitude': 'float64',
            'latitude': 'float64',
            'phone': 'string',
            'website_url': 'string',
            'state': 'string',
            'street': 'string'
        })

In [ ]:
def process_date(df):
    # Get the current date in 'YYYYMMDD' format
    format_dt = datetime.now().strftime('%Y%m%d')
    # Add the processing date column
    df['process_date'] = format_dt
    return df

def clean_columns(df):
    df.columns = (
        df.columns
        .str.lower()         # Convert column names to lowercase
        .str.replace(' ', '_')  # Replace spaces with underscores
        .str.strip()         # Remove leading and trailing whitespace
    )
    return df

def clean_partitions(s):
    if isinstance(s, str):
        s = s.strip()                   # Remove espaços extras no início e no final
        # transforma caracteres acentuados em sua forma decomposta, permitindo a remoção dos acentos.
        # identifica e remove os acentos, mantendo apenas as letras básicas.
        s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))  # Remove acentos
        s = s.replace(" ", "_")         # Substitui espaços internos por underscores
    return s

In [210]:
df = process_date(df_read)
# Clean the DataFrame columns
dfFinal = clean_columns(df)

print(dfFinal.isna().sum())
#dfFinal
dfAddres = dfFinal[dfFinal['address_1'].isna() & dfFinal['address_3'].notna()]
dfAddres

id                   0
name                 0
brewery_type         0
address_1          764
address_2         8247
address_3         8320
city                 0
state_province       0
postal_code          0
country              0
longitude         2328
latitude          2328
phone              851
website_url       1189
state                0
street             764
process_date         0
dtype: int64


,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street,process_date


In [122]:
dfAddres = dfFinal[dfFinal['country'] == 'Austria' ]
dfAddres

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street,process_date
1610,b04c9621-bb87-439d-a499-94cb432bde90,Cafe Luis,bar,Weizberg 7,<NA>,<NA>,Weiz,Steiermark,8160,Austria,15.635828,47.223789,+4331723924,<NA>,Steiermark,Weizberg 7,20241108
1612,9355bc7c-bdc7-40c3-8932-e840c996ed86,Caf� Okei,bar,Feldkirchenstra�e 40,<NA>,<NA>,Kalsdorf bei Graz,Steiermark,8401,Austria,15.460420,46.971319,+43313557156,https://okei.at,Steiermark,Feldkirchenstra�e 40,20241108
2324,16fb2c6e-c7e1-4d46-b9d0-1c86816eb129,Die Bierbotschaft,brewpub,Ponigler Str. 52,<NA>,<NA>,Wundschuh,Steiermark,8142,Austria,15.455187,46.912916,+436763530560,https://www.bierbotschaft.at,Steiermark,Ponigler Str. 52,20241108
3250,fcd9312c-7531-4801-b481-52f4267cf628,Gösser Brauerei (Heineken),large,Brauhausgasse 1,<NA>,<NA>,Leoben,Steiermark,8700,Austria,15.093344,47.362557,+438102069791,https://www.goesser.at,Steiermark,Brauhausgasse 1,20241108
4204,867eee9e-2138-4e62-97a9-b25e7fd1a1ff,Klosterbrauerei Stift Engelszell,nano,Stiftstra�e 6,<NA>,<NA>,Engelhartszell,Oberösterreich,4090,Austria,13.734607,48.498616,+4377178010,https://www.stift-engelszell.at/trappistenbrau...,Oberosterreich,Stiftstra�e 6,20241108
5098,524ab865-1df1-4655-b19f-0eb427cfa0d6,Murauer Brauerei,large,Raffaltpl. 19,<NA>,<NA>,Murau,Steiermark,8850,Austria,14.165061,47.111016,+43353232660,https://www.murauerbier.at,Steiermark,Raffaltpl. 19,20241108
5913,56758247-521c-4219-afa2-0665c48c50ef,Puntigamer Brauerei (Heineken),large,Triester Str. 361,<NA>,<NA>,Graz,Steiermark,8055,Austria,15.432570,47.029628,+4331629710013,https://www.puntigamer.at,Steiermark,Triester Str. 361,20241108
6488,eb82907e-7f0d-4453-b07e-4e00763e1cf6,Schladminger Brauerei (Heineken),large,Hammerfeldweg 163,<NA>,<NA>,Schladming,Steiermark,8970,Austria,13.685049,47.390297,+433687225910,https://www.schladmingerbier.at,Steiermark,Hammerfeldweg 163,20241108
6492,c88759b6-88d6-4172-bed7-61dec61e50aa,Schleppe Brauerei (Heineken),large,Schleppe-Platz 1,<NA>,<NA>,Klagenfurt am W�rthersee,K�rnten,9020,Austria,14.286632,46.641697,+4346342700,http://www.schleppe.at,K�rnten,Schleppe-Platz 1,20241108
6500,4c8dff75-fa8c-41ce-9999-1f063b59addb,Schwechater Brauerei (Heineken),large,Mautner-Markhof-Stra�e 11,<NA>,<NA>,Schwechat,Nieder�sterreich,2320,Austria,16.467173,48.144924,+431701400,https://www.schwechater.at,Nieder�sterreich,Mautner-Markhof-Stra�e 11,20241108


In [78]:
duplicados = dfFinal[dfFinal.duplicated()]

duplicados

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street,process_date


In [211]:
df = dfFinal

partition_colums = ['country']

for col in partition_colums:
    df[col] = df[col].apply(clean_partitions)

df

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street,process_date
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,micro,1716 Topeka St,<NA>,<NA>,Norman,Oklahoma,73069-8224,United_States,-97.468182,35.257389,4058160490,http://www.405brewing.com,Oklahoma,1716 Topeka St,20241111
1,9c5a66c8-cc13-416f-a5d9-0a769c87d318,(512) Brewing Co,micro,407 Radam Ln Ste F200,<NA>,<NA>,Austin,Texas,78745-1197,United_States,NaN,NaN,5129211545,http://www.512brewing.com,Texas,407 Radam Ln Ste F200,20241111
2,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,1 of Us Brewing Company,micro,8100 Washington Ave,<NA>,<NA>,Mount Pleasant,Wisconsin,53406-3920,United_States,-87.883364,42.720108,2624847553,https://www.1ofusbrewing.com,Wisconsin,8100 Washington Ave,20241111
3,ef970757-fe42-416f-931d-722451f1f59c,10 Barrel Brewing Co,large,1501 E St,<NA>,<NA>,San Diego,California,92101-6618,United_States,-117.129593,32.714813,6195782311,http://10barrel.com,California,1501 E St,20241111
4,6d14b220-8926-4521-8d19-b98a2d6ec3db,10 Barrel Brewing Co,large,62970 18th St,<NA>,<NA>,Bend,Oregon,97701-9847,United_States,-121.281706,44.086835,5415851007,http://www.10barrel.com,Oregon,62970 18th St,20241111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8350,57a70f5c-a995-4be1-949c-99244450940f,플레이그라운드 브루어리(playground Brewery),brewpub,"246-13, Isanpo-gil",Ilsanseo-gu,<NA>,Goyang-si,Gyeonggido,10203,South_Korea,126.702011,37.667795,031-912-2463,https://www.playgroundbrewery.com/,Gyeonggido,"246-13, Isanpo-gil",20241111
8351,64a4f65c-47a9-4d8e-ad2e-3cc898b9689c,핸드앤몰트 (Hand and Malt),brewpub,"19, Hangang-daero 38ga-gil",<NA>,<NA>,Yongsan-gu,Seoul,04386,South_Korea,126.970583,37.529857,070-7178-4011,http://handandmalt.com/,Seoul,"19, Hangang-daero 38ga-gil",20241111
8352,a9541aa9-5c12-469b-8cbb-01fb5619cefc,허심청브루잉(Hursimchung Bru),brewpub,"23, Geumganggongwon-ro 20beon-gil",<NA>,<NA>,Dongnae-gu,Busan,47709,South_Korea,129.082603,35.219902,051-550-2345,https://www.hotelnongshim.com/kr/index.php?pCo...,Busan,"23, Geumganggongwon-ro 20beon-gil",20241111
8353,6f317bdc-458e-466f-bd2a-9ab398d46631,헤이스탁 브루어리(Haystack Brewery),brewpub,"25, Pangyogongwon-ro 3-gil",Bundang-gu,<NA>,Seongnam-si,Gyeonggido,13477,South_Korea,127.089061,37.391256,070-7631-1110,<NA>,Gyeonggido,"25, Pangyogongwon-ro 3-gil",20241111


In [212]:
#format_dt = datetime.now().strftime('%Y%m%d')
#path = f"datalake/silver/{format_dt}/"
# path = "datalake/silver/"
file_name = "breweries.parquet"

        # Combine the path and filename
#file_path = path + file_name


        # Save the DataFrame as a parquet file with partitions and compression
df.to_parquet(
    file_name, partition_cols=partition_colums, compression='gzip', engine='pyarrow'
)

In [ ]:
table = pq.read_table('breweries.parquet', columns=['id', 'country', 'state', 'city', 'brewery_type'])
df = table.to_pandas()

df

,id,country,state,city,brewery_type
0,b04c9621-bb87-439d-a499-94cb432bde90,Austria,Steiermark,Weiz,bar
1,9355bc7c-bdc7-40c3-8932-e840c996ed86,Austria,Steiermark,Kalsdorf bei Graz,bar
2,16fb2c6e-c7e1-4d46-b9d0-1c86816eb129,Austria,Steiermark,Wundschuh,brewpub
3,fcd9312c-7531-4801-b481-52f4267cf628,Austria,Steiermark,Leoben,large
4,867eee9e-2138-4e62-97a9-b25e7fd1a1ff,Austria,Oberosterreich,Engelhartszell,nano
...,...,...,...,...,...
8350,a602e9ea-f0b3-4a40-8aa8-02e0aace191c,United_States,Colorado,Fort Collins,micro
8351,8d18da7a-1ed9-4eb4-b87c-ac444aa81e14,United_States,Florida,Tampa,brewpub
8352,aabc26d3-4870-42e9-ae09-1333f85c3246,United_States,Colorado,Littleton,micro
8353,82234375-c7fa-4208-8878-b661b7a1e6aa,United_States,South_Dakota,Rapid City,micro


In [ ]:
import pandas as pd
import pyarrow.parquet as pq

# Caminho do diretório onde estão as partições por país
dataset_path = "breweries.parquet"

# Lista para armazenar os DataFrames agregados de cada país
dfs = []

# Carregar o dataset particionado e obter a lista de valores únicos na coluna 'pais'
dataset = pq.ParquetDataset(dataset_path, use_legacy_dataset=False)

for fragment in dataset.fragments:
    # Ler a partição específica como um PyArrow Table
    table = fragment.to_table()
    df = table.to_pandas()
    #print(fragment)
    partition = (str(fragment).split('/')[1]).split('=')[1]
    #print(partition)
    #print(part.split('=')[1])
    c = partition
    print(fragment)
    df['country'] = c
    # Realizar a agregação para o país atual, contando `id` únicos para cada combinação
    df_agg = (
        df.groupby([ 'country', 'state', 'city', 'brewery_type'])['id']
        .count().reset_index(name='number_of_breweries')
    )
    
    # Adicionar o DataFrame agregado à lista
    dfs.append(df_agg)

# Concatenar todos os DataFrames agregados em um único DataFrame final
df_final = pd.concat(dfs, ignore_index=True)

# Salvar o resultado agregado final em um novo Parquet ou CSV
# df_final.to_parquet("path/to/output/final_aggregated_view.parquet")
df_final

C:\Users\Bhrenner\AppData\Local\Temp\ipykernel_26812\310381616.py:11: FutureWarning: Passing 'use_legacy_dataset' is deprecated as of pyarrow 15.0.0 and will be removed in a future version.
  dataset = pq.ParquetDataset(dataset_path, use_legacy_dataset=False)


<pyarrow.dataset.ParquetFileFragment path=breweries.parquet/country=Austria/236d2c2e7e7346569c75e68913a48c82-0.parquet partition=[country=Austria]>
<pyarrow.dataset.ParquetFileFragment path=breweries.parquet/country=England/236d2c2e7e7346569c75e68913a48c82-0.parquet partition=[country=England]>
<pyarrow.dataset.ParquetFileFragment path=breweries.parquet/country=France/236d2c2e7e7346569c75e68913a48c82-0.parquet partition=[country=France]>
<pyarrow.dataset.ParquetFileFragment path=breweries.parquet/country=Germany/236d2c2e7e7346569c75e68913a48c82-0.parquet partition=[country=Germany]>
<pyarrow.dataset.ParquetFileFragment path=breweries.parquet/country=Ireland/236d2c2e7e7346569c75e68913a48c82-0.parquet partition=[country=Ireland]>
<pyarrow.dataset.ParquetFileFragment path=breweries.parquet/country=Isle_of_Man/236d2c2e7e7346569c75e68913a48c82-0.parquet partition=[country=Isle_of_Man]>
<pyarrow.dataset.ParquetFileFragment path=breweries.parquet/country=Poland/236d2c2e7e7346569c75e68913a48c8

,country,state,city,brewery_type,number_of_breweries
0,Austria,K�rnten,Klagenfurt am W�rthersee,large,1
1,Austria,K�rnten,Villach,large,1
2,Austria,K�rnten,Wimitz,nano,1
3,Austria,Nieder�sterreich,Schwechat,large,1
4,Austria,Nieder�sterreich,Wieselburg,large,1
...,...,...,...,...,...
5136,United_States,Wyoming,Saratoga,brewpub,1
5137,United_States,Wyoming,Sheridan,micro,3
5138,United_States,Wyoming,Ten Sleep,micro,1
5139,United_States,Wyoming,Thermopolis,brewpub,1


In [201]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os

# Função para criptografar uma palavra e gerar um arquivo
def criptografar_palavra(palavra, chave, iv, nome_arquivo):
    padder = padding.PKCS7(128).padder()
    palavra_padded = padder.update(palavra.encode()) + padder.finalize()
    
    # Criptografar a palavra
    cipher = Cipher(algorithms.AES(chave), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(palavra_padded) + encryptor.finalize()

    
    # Grava o IV e o texto criptografado no arquivo de saída
    with open(arquivo_saida, 'wb') as f:
        f.write(iv)  # O IV é armazenado no início
        f.write(ciphertext)  # O texto criptografado

    print(f"Palavra criptografada com sucesso! Arquivo gerado: {arquivo_saida}")

def descriptografar_palavra(nome_arquivo, chave):
    # Ler o IV e o texto criptografado do arquivo
    with open(nome_arquivo, "rb") as f:
        iv = f.read(16)  # O IV tem 16 bytes (128 bits)
        ciphertext = f.read()  # O restante é o texto criptografado

    # Descriptografar o texto
    cipher = Cipher(algorithms.AES(chave), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    palavra_padded = decryptor.update(ciphertext) + decryptor.finalize()

    # Remover o padding corretamente
    try:
        unpadder = padding.PKCS7(128).unpadder()
        palavra_original = unpadder.update(palavra_padded) + unpadder.finalize()
        return palavra_original.decode()
    except ValueError as e:
        print("Erro ao remover padding:", e)
        return None



chave_fixa = b"12345678901234567890123456789012"  # 32 bytes fixos (256 bits)
iv_fixo = b"abcdefghijklmnop"  # 16 bytes fixos (128 bits)
palavra = "breCase#072024"
    
    # Caminho do arquivo onde será salvo o texto criptografado
arquivo_saida = 'password.txt'
    
    # Criptografando a palavra e gerando o arquivo
criptografar_palavra(palavra, chave_fixa, iv_fixo, arquivo_saida)

palavra_original = descriptografar_palavra(arquivo_saida, chave)

if palavra_original:
        print(f"Palavra original descriptografada: {palavra_original}")

Palavra criptografada com sucesso! Arquivo gerado: password.txt
Erro ao remover padding: Invalid padding bytes.


In [203]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

# Função para gerar uma chave fixa e um IV fixo
def gerar_chave_iv_fixos():
    # Chave fixa de 256 bits (32 bytes) e IV fixo de 128 bits (16 bytes)
    chave_fixa = b"12345678901234567890123456789012"  # 32 bytes fixos (256 bits)
    iv_fixo = b"abcdefghijklmnop"  # 16 bytes fixos (128 bits)
    return chave_fixa, iv_fixo

# Função para criptografar uma palavra e salvar no arquivo
def criptografar_em_arquivo(palavra, chave, iv, nome_arquivo):
    # Preparar a palavra com padding
    padder = padding.PKCS7(128).padder()
    palavra_padded = padder.update(palavra.encode()) + padder.finalize()
    
    # Criptografar a palavra
    cipher = Cipher(algorithms.AES(chave), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(palavra_padded) + encryptor.finalize()

    # Salvar o IV e o texto criptografado no arquivo
    with open(nome_arquivo, "wb") as f:
        f.write(iv)  # Escrever o IV
        f.write(ciphertext)  # Escrever o texto criptografado
    print(f"Texto criptografado salvo em {nome_arquivo}")

# Função para descriptografar o conteúdo do arquivo e recuperar a palavra
def descriptografar_de_arquivo(nome_arquivo, chave):
    try:
        # Ler o IV e o texto criptografado do arquivo
        with open(nome_arquivo, "rb") as f:
            iv = f.read(16)  # O IV tem 16 bytes (128 bits)
            ciphertext = f.read()  # O restante é o texto criptografado

        # Verificar se a quantidade de dados lidos está correta
        if len(ciphertext) == 0:
            raise ValueError("Arquivo de criptografia está vazio ou corrompido.")

        # Descriptografar o texto
        cipher = Cipher(algorithms.AES(chave), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()
        palavra_padded = decryptor.update(ciphertext) + decryptor.finalize()

        # Remover o padding corretamente
        unpadder = padding.PKCS7(128).unpadder()
        palavra_original = unpadder.update(palavra_padded) + unpadder.finalize()
        return palavra_original.decode()

    except ValueError as e:
        print("Erro ao remover padding:", e)
        return None
    except Exception as e:
        print(f"Erro inesperado: {e}")
        return None

# Exemplo de uso
if __name__ == "__main__":
    # Palavra a ser criptografada
    palavra = "breCase#072024"
    
    # Gerar chave fixa e IV fixo
    chave_fixa, iv_fixo = gerar_chave_iv_fixos()
    
    # Nome do arquivo onde o texto criptografado será salvo
    arquivo_saida = "password.txt"
    
    # Criptografar a palavra e salvar no arquivo
    criptografar_em_arquivo(palavra, chave_fixa, iv_fixo, arquivo_saida)
    
    # Descriptografar o arquivo e recuperar a palavra original
    palavra_recuperada = descriptografar_de_arquivo(arquivo_saida, chave_fixa)
    
    if palavra_recuperada:
        # Exibir a palavra recuperada
        print(f"Palavra original recuperada do arquivo: {palavra_recuperada}")
    else:
        print("Erro ao descriptografar o arquivo.")


Texto criptografado salvo em password.txt
Palavra original recuperada do arquivo: breCase#072024


In [4]:
import pandas as pd
import pyarrow.parquet as pq

# Caminho do diretório onde estão as partições por país
dataset_path = "BreweriesbyLocType"


dfG = pd.read_parquet(dataset_path)

dfG 
 

,country,state,city,brewery_type,number_of_breweries,process_date
0,Austria,K�rnten,Klagenfurt am W�rthersee,large,1,20241111
1,Austria,K�rnten,Villach,large,1,20241111
2,Austria,K�rnten,Wimitz,nano,1,20241111
3,Austria,Nieder�sterreich,Schwechat,large,1,20241111
4,Austria,Nieder�sterreich,Wieselburg,large,1,20241111
...,...,...,...,...,...,...
5137,United_States,Wyoming,Saratoga,brewpub,1,20241111
5138,United_States,Wyoming,Sheridan,micro,3,20241111
5139,United_States,Wyoming,Ten Sleep,micro,1,20241111
5140,United_States,Wyoming,Thermopolis,brewpub,1,20241111


In [191]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

# Função para criptografar a palavra usando uma chave e IV fixos
def criptografar_palavra(palavra, chave, iv):
    # Preparar a palavra com padding
    padder = padding.PKCS7(128).padder()
    palavra_padded = padder.update(palavra.encode()) + padder.finalize()
    
    # Criptografar a palavra
    cipher = Cipher(algorithms.AES(chave), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(palavra_padded) + encryptor.finalize()
    
    return ciphertext

# Função para descriptografar a palavra usando a chave e IV fixos
def descriptografar_palavra(iv, ciphertext, chave):
    # Inicializar o objeto Cipher com o IV e a chave
    cipher = Cipher(algorithms.AES(chave), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    
    # Descriptografar o texto
    palavra_padded = decryptor.update(ciphertext) + decryptor.finalize()
    
    # Remover o padding
    unpadder = padding.PKCS7(128).unpadder()
    palavra_original = unpadder.update(palavra_padded) + unpadder.finalize()
    
    return palavra_original.decode()

# Chave e IV fixos (defina valores específicos para ambos)
# A chave deve ser de 32 bytes (256 bits) para AES-256
chave_fixa = b"12345678901234567890123456789012"  # 32 bytes fixos (256 bits)
# O IV deve ser de 16 bytes (128 bits) para AES
iv_fixo = b"abcdefghijklmnop"  # 16 bytes fixos (128 bits)

ciphertext = criptografar_palavra(palavra, chave_fixa, iv_fixo)
print(f"Texto criptografado: {ciphertext.hex()}")
    
# Descriptografar a palavra
palavra_descriptografada = descriptografar_palavra(iv_fixo, ciphertext, chave_fixa)
print(f"Palavra original descriptografada: {palavra_descriptografada}")


Texto criptografado: 67b1ae31f94fa8546654784bea7599cb
Palavra original descriptografada: Segredo
